In [1]:
import pandas as pd
import numpy as np
import os

In [25]:
base_path="/net/mraid08/export/jafar/UKBioBank/Data/ukb29741_a1c_below_65"   
files_suffixes=["_train.csv","_val.csv","_train_val.csv","_test.csv","_updates_scoreboard_train.csv",
                "_updates_scoreboard_val.csv", "_updates_scoreboard_train_val.csv",
                "_updates_scoreboard_test.csv"]

In [3]:
diab_age_data_path="/net/mraid08/export/jafar/UKBioBank/Data/ukb29741.csv"
diab_data_col=pd.read_csv(diab_age_data_path, nrows=0).columns.values

In [4]:
diab_age_col = [x for x in diab_data_col if x.startswith("2976-")]  # Aged when diabetes first diagnosed

In [5]:
Diab_age_df = pd.read_csv(diab_age_data_path, usecols=diab_age_col + ["eid"], index_col="eid")
Diab_age_df["Min_diab_age"] = Diab_age_df.min(axis=1)

In [10]:
Diab_age_df.head()

,2976-0.0,2976-1.0,2976-2.0,Min_diab_age
eid,,,,
4704502,NaN,NaN,NaN,NaN
4020781,NaN,NaN,NaN,NaN
5473158,NaN,NaN,NaN,NaN
3379784,NaN,NaN,NaN,NaN
2014430,NaN,NaN,NaN,NaN


In [32]:
def calc_TTE(row):
    """
    Returns either the time between the first visit to the first appearance of diabetes, or
    if diabetes was not diagnosed, or the time of diagnosis is not given - return the time of last visit"""
    if pd.isnull(row["TTE"]):
        return row["21003-4.0"]
    else:
        return row["TTE"] 

In [27]:
data_dict_save={}
data_dict_drop={}
orig_data={}
for path_suffix in files_suffixes:
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print(path_suffix)
    short_data_path=base_path+path_suffix
    dropped_data_path=base_path+"_early_T2D_detection_"+path_suffix
    short_data=pd.read_csv(short_data_path,index_col="eid")
    orig_data[path_suffix]=short_data
    print("short_data.shape:",short_data.shape)
    age_diab_cols=[x for x in short_data.columns if x.startswith("2976")]
    if len(age_diab_cols)==0:
        tmp=short_data.join(Diab_age_df,how="left")
        tmp["TTE"]= tmp["Min_diab_age"] - tmp["21003-0.0"]
        tmp_drop_index=tmp.loc[tmp["TTE"]<=0,:].index
        tmp_save_index=[x for x in tmp.index if x not in tmp_drop_index]
        tmp_drop=tmp.loc[tmp_drop_index,:]
        tmp_drop.to_csv(dropped_data_path,index="True")
        tmp_save=tmp.loc[tmp_save_index,:]
        tmp_save.to_csv(short_data_path,index="True")
        
        data_dict_save[path_suffix]=tmp_save
        data_dict_drop[path_suffix]=tmp_drop
        
        print("tmp_drop.shape:",tmp_drop.shape)
        print("tmp_save.shape:",tmp_save.shape)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
_train.csv
short_data.shape: (25122, 1236)
tmp_drop.shape: (97, 1241)
tmp_save.shape: (25025, 1241)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
_val.csv
short_data.shape: (10757, 1236)
tmp_drop.shape: (33, 1241)
tmp_save.shape: (10724, 1241)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
_train_val.csv
short_data.shape: (35879, 1236)
tmp_drop.shape: (130, 1241)
tmp_save.shape: (35749, 1241)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
_test.csv
short_data.shape: (8994, 1236)
tmp_drop.shape: (34, 1241)
tmp_save.shape: (8960, 1241)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
_updates_scoreboard_train.csv
short_data.shape: (25122, 1320)
tmp_drop.shape: (97, 1325)
tmp_save.shape: (25025, 1325)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
_updates_scoreboard_val.csv
short_data.shape: (10757, 1320)
tmp_drop.shape: (33, 1325)
tmp_save.shape: 

In [34]:
data_dict_save.keys()

dict_keys([])

In [38]:
data_dict_save={}
data_dict_drop={}
orig_data={}
for path_suffix in files_suffixes:
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print(path_suffix)
    short_data_path=base_path+path_suffix
    tmp=pd.read_csv(short_data_path,index_col="eid")
    tmp["TTE"]=tmp.apply(calc_TTE,axis=1)
    print(tmp.head())
    print(tmp["TTE"].head())
    print(tmp.shape)
    tmp.to_csv(short_data_path,index="True")

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
_train.csv
         21003-3.0  21003-4.0  46-0.0  47-0.0  48-0.0  49-0.0  50-0.0  \
eid                                                                     
5042569       63.0        9.0    16.0    16.0   116.0   118.0   168.0   
1377679       50.0        7.0    40.0    44.0    81.0    96.0   166.0   
5892004       64.0        3.0    19.0    19.0    90.0   110.0   165.0   
5591625       57.0        7.0    18.0    25.0    71.0    91.0   165.0   
4608302       69.0        5.0    18.0    16.0    78.0   109.0   160.0   

           78-0.0  102-0.0  189-0.0  ...  HDL cholesterol_woe  \
eid                                  ...                        
5042569 -0.242680     72.0 -2.96255  ...             0.223226   
1377679 -0.046000     47.0 -6.07871  ...            -0.511755   
5892004 -0.242858     68.0 -3.76781  ...            -1.888472   
5591625 -0.242801     65.0  3.45470  ...            -0.967802   
4608302 -0.242407     67

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
_updates_scoreboard_train.csv
         21003-3.0  21003-4.0  46-0.0  47-0.0  48-0.0  49-0.0  50-0.0  \
eid                                                                     
5042569       63.0        9.0    16.0    16.0   116.0   118.0   168.0   
1377679       50.0        7.0    40.0    44.0    81.0    96.0   166.0   
5892004       64.0        3.0    19.0    19.0    90.0   110.0   165.0   
5591625       57.0        7.0    18.0    25.0    71.0    91.0   165.0   
4608302       69.0        5.0    18.0    16.0    78.0   109.0   160.0   

           78-0.0  102-0.0  189-0.0  ...  Triglycerides > 0.89 and <= 1.2  \
eid                                  ...                                    
5042569 -0.242680     72.0 -2.96255  ...                                0   
1377679 -0.046000     47.0 -6.07871  ...                                1   
5892004 -0.242858     68.0 -3.76781  ...                                0   
5591625 -0

In [29]:
tmp_val=pd.read_csv(
    "/net/mraid08/export/jafar/UKBioBank/Data/ukb29741_a1c_below_65_updates_scoreboard_val.csv")

In [31]:
tmp_val.loc[:,"21003-4.0"]

0         7.0
1         7.0
2         5.0
3         8.0
4         6.0
         ... 
10719     5.0
10720    10.0
10721     4.0
10722     8.0
10723     8.0
Name: 21003-4.0, Length: 10724, dtype: float64

In [ ]:
data_col = pd.read_csv(diab_age_data_path, nrows=0).columns.values
diab_age_col = [x for x in diab_data_col if x.startswith("2976-")]